## Load the data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


Obj_Geodyn = {}
for i,val in enumerate(['msis2']):
    run_settings = "/data/geodyn_proj/validation/NoCD_Arc36_AllDensModels_settings/NoCdArc36_msis2.yaml"
    
    ### Load the data into an object
    Obj_Geodyn[val] = Pygeodyn(run_settings)
    Obj_Geodyn[val].getData()

    

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px

import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

# Simplify Plotting Schemes:
col1 = px.colors.qualitative.Plotly[0]
col2 = px.colors.qualitative.Plotly[1]
col3 = px.colors.qualitative.Plotly[2]
col4 = px.colors.qualitative.Plotly[3]
col5 = px.colors.qualitative.Plotly[4]
col6 = px.colors.qualitative.Plotly[5]

In [ ]:
# %load_ext autoreload
# %autoreload 2


# from PYGEODYNAnalysis_icesat2_validationwork import  PLOT__intrack_residuals_w_rms
# fig = make_subplots(
#     rows=1, cols=1,
#     subplot_titles=(['In Track Residuals']),
#     vertical_spacing = 0.1)

# fig = PLOT__intrack_residuals_w_rms(fig,  Obj_Geodyn['msis2'], 'msis2'  , 0 )

# fig.show(config=config)


In [ ]:
obj_m1 = Obj_Geodyn['msis2']

## Load NTW residuals 
Also make coordinate transformations to R theta phi

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys
sys.path.insert(0,'/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/')
from common_functions          import Convert_cartesian_to_RSW, Convert_cartesian_to_NTW_getT, Convert_cartesian_to_NTW_returnall


###### GET THE PCE DATA:
StateVector_PCE_datafile = '/data/data_geodyn/inputs/icesat2/setups/PCE_ascii.txt'
SAT_ID = int(obj_m1.__dict__['global_params']['SATID'])
which_stat = 'CURRENT_VALUE'

for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):#[::2]):
    print(arc)

    ####--------------------- Residual  ---------------------

    arc_first_time  = obj_m1.__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC'].iloc[0]
    arc_last_time   = obj_m1.__dict__['Trajectory_orbfil'][arc]['data_record']['Date_UTC'].iloc[-2]

    arc_first_time_str     =  str(arc_first_time)#.replace( "'",' ') 
    arc_last_time_str      =  str(arc_last_time)#.replace( "'",' ') 


    A=[]
    for i,val in enumerate(np.arange(-20,20)):
        A.append(str(pd.to_datetime(arc_first_time)+pd.to_timedelta(val,'s')))
    B=[]
    for i,val in enumerate(np.arange(-20,20)):
        B.append(str(pd.to_datetime(arc_last_time)+pd.to_timedelta(val,'s')))

    ####---------------------------------------------------------
    last_line = False
    with open(StateVector_PCE_datafile, 'r') as f:
        for line_no, line_text in enumerate(f):
            if any(times in line_text for times in A):
                first_line = line_no
            if any(times in line_text for times in B):
                last_line = line_no
                break

        if not last_line:
            last_line = first_line +32220
            print('No matching lastline time: ',arc_last_time_str, last_line )

    ####   IF YOU GET AN ERROR HERE stating that either first_line or last_line is 
    ####    It is probably an issue with the date in the arc not matching up with the dates given in the PCEfile
    print('Loading PCE data...')
    PCE_data = pd.read_csv(StateVector_PCE_datafile, 
                skiprows = first_line, 
                nrows=last_line-first_line,           
                sep = '\s+',
                dtype=str,
                names = [
                        'Date',
                        'MJDSECs', 
                        'RSECS', #(fractional secs)
                        'GPS offset', # (UTC - GPS offset (secs))
                        'X',
                        'Y',
                        'Z',
                        'X_dot',
                        'Y_dot',
                        'Z_dot',
                        'YYMMDDhhmmss',
                            ],)

    PCE_data['Date_pd'] = pd.to_datetime(PCE_data['Date'])
    orbfil_arc1 = obj_m1.__dict__['Trajectory_orbfil'][arc]['data_record']
    orbfil_arc1['Date_pd'] = pd.to_datetime(orbfil_arc1 ['Date_UTC'])


    ### C_1 is a dataframe containing all data between the two files where the dates match
    C_1 = pd.merge(left=orbfil_arc1, left_on='Date_pd',
         right=PCE_data, right_on='Date_pd')



In [ ]:


### Convert the ORBIT FILE data to NTW
print('Converting OrbitFile data to other coordinates...')

data_orbfil = {}
data_PCE    = {}
resids      = {}

X = C_1['Satellite Inertial X coordinate']
Y = C_1['Satellite Inertial Y coordinate']
Z = C_1['Satellite Inertial Z coordinate']
Xdot = C_1['Satellite Inertial X velocity']
Ydot = C_1['Satellite Inertial Y velocity']
Zdot = C_1['Satellite Inertial Z velocity']
state_vector = np.transpose(np.array([X, Y, Z, Xdot, Ydot, Zdot]))
data_orbfil['Date'] = C_1['Date_pd']

##### NTW Coordinate System
NTW_orbfil  = [Convert_cartesian_to_NTW_returnall(x) for x in state_vector]
data_orbfil['N'] = np.array(NTW_orbfil)[:,0]
data_orbfil['T'] = np.array(NTW_orbfil)[:,1]
data_orbfil['W'] = np.array(NTW_orbfil)[:,2]
##### XYZ Coordinate System
data_orbfil['X'] = X
data_orbfil['Y'] = Y
data_orbfil['Z'] = Z
##### R theta phi Coordinate System
data_orbfil['R']     = np.sqrt( np.square(X) + 
                                np.square(Y) +
                                np.square(Z) )
data_orbfil['theta'] = np.arctan(Y / X)
data_orbfil['phi']   = np.arccos(Z / (np.sqrt( np.square(X) + 
                                np.square(Y) +
                                np.square(Z) )))


### Convert the PCE data to NTW
print('Converting PCE data to other coordinates...')
X = C_1['X'].astype(float)
Y = C_1['Y'].astype(float)
Z = C_1['Z'].astype(float)
Xdot = C_1['X_dot'].astype(float)
Ydot = C_1['Y_dot'].astype(float)
Zdot = C_1['Z_dot'].astype(float)
state_vector = np.transpose(np.array([X, Y, Z, Xdot, Ydot, Zdot]))
data_PCE['Date'] = C_1['Date_pd']

##### NTW Coordinate System
NTW_PCE  = [Convert_cartesian_to_NTW_returnall(x) for x in state_vector]
data_PCE['N'] = np.array(NTW_PCE)[:,0]
data_PCE['T'] = np.array(NTW_PCE)[:,1]
data_PCE['W'] = np.array(NTW_PCE)[:,2]
##### XYZ Coordinate System
data_PCE['X'] = X
data_PCE['Y'] = Y
data_PCE['Z'] = Z
##### R theta phi Coordinate System
data_PCE['R']     = np.sqrt( np.square(X) + 
                                np.square(Y) +
                                np.square(Z) )
data_PCE['theta'] = np.arctan(Y / X)
data_PCE['phi']   = np.arccos(Z / (np.sqrt( np.square(X) + 
                                np.square(Y) +
                                np.square(Z) )))


### RESIDUALS:
resids['Date'] = C_1['Date_pd']

##### NTW Coordinate System
resids['N'] = (np.array(data_PCE['N']) - np.array(data_orbfil['N']))
resids['T'] = (np.array(data_PCE['T']) - np.array(data_orbfil['T']))
resids['W'] = (np.array(data_PCE['W']) - np.array(data_orbfil['W']))
##### XYZ Coordinate System
resids['X'] = (np.array(data_PCE['X']) - np.array(data_orbfil['X']))
resids['Y'] = (np.array(data_PCE['Y']) - np.array(data_orbfil['Y']))
resids['Z'] = (np.array(data_PCE['Z']) - np.array(data_orbfil['Z']))
##### R theta phi Coordinate System
resids['R']     = (np.array(data_PCE['R'])     - np.array(data_orbfil['R']))
resids['theta'] = (np.array(data_PCE['theta']) - np.array(data_orbfil['theta']))
resids['phi']   = (np.array(data_PCE['phi'])   - np.array(data_orbfil['phi']))





## Plot Modeled Orbits

### XYZ 

In [ ]:
coords = ['X','Y','Z']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.2)
fig.update_layout(title=''.join(coords)+' Orbit',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=data_orbfil['Date'],
                             y=data_orbfil[val],
                         name=val+' OrbFil',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)
    fig.add_trace(go.Scattergl(
                             x=data_PCE['Date'],
                             y=data_PCE[val],
                         name=val+' PCE',
                         mode='markers',
                         marker=dict(
                             color=col2,
                             size=ms_size-2,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="date"    ,row=3, col=1)

fig.show(config=config)


### R theta phi

In [ ]:
coords = ['R','theta','phi']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.2)
fig.update_layout(title=''.join(coords)+' Orbit',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=data_orbfil['Date'],
                             y=data_orbfil[val],
                         name=val+' OrbFil',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)
    fig.add_trace(go.Scattergl(
                             x=data_PCE['Date'],
                             y=data_PCE[val],
                         name=val+' PCE',
                         mode='markers',
                         marker=dict(
                             color=col2,
                             size=ms_size-2,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="degree [rad]" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="degree [rad]" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="date"    ,row=3, col=1)

fig.show(config=config)



### NTW

In [ ]:
coords = ['N','T','W']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.)
fig.update_layout(title=''.join(coords)+' Orbit',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=data_orbfil['Date'],
                             y=data_orbfil[val],
                         name=val+' OrbFil',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)
    fig.add_trace(go.Scattergl(
                             x=data_PCE['Date'],
                             y=data_PCE[val],
                         name=val+' PCE',
                         mode='markers',
                         marker=dict(
                             color=col2,
                             size=ms_size-2,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="date"    ,row=3, col=1)

fig.show(config=config)


## Plot Residuals in diff coords

### XYZ

In [ ]:
coords = ['X','Y','Z']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.2)
fig.update_layout(title=  'Residuals '+  ''.join(coords)+' System',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=resids['Date'],
                             y=resids[val],
                         name=val+' resids',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="Date"    ,row=3, col=1)

fig.show(config=config)


### NTW

In [ ]:
coords = ['N','T','W']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.2)
fig.update_layout(title=  'Residuals '+  ''.join(coords)+' System',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=resids['Date'],
                             y=resids[val],
                         name=val+' resids',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="Date"    ,row=3, col=1)

fig.show(config=config)


### R theta phi

In [ ]:
coords = ['R','theta','phi']
ms_size = 5

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(coords),
    vertical_spacing = 0.2)
fig.update_layout(title=  'Residuals '+  ' '.join(coords)+' System',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

for i,val in enumerate(coords):
    fig.add_trace(go.Scattergl(
                             x=resids['Date'],
                             y=resids[val],
                         name=val+' resids',
                         mode='markers',
                         marker=dict(
                             color=col1,
                             size=ms_size,
                                    ),
                         showlegend=True),
                         secondary_y=False,
                         row=i+1, col=1)

fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="degree [rad]" ,type="linear" , exponentformat= 'power',row=2, col=1)
fig.update_yaxes( title="degree [rad]" ,type="linear" , exponentformat= 'power',row=3, col=1)
fig.update_xaxes( title="Date"    ,row=3, col=1)

fig.show(config=config)


## Spectrogram

In [ ]:
# from scipy import signal
# f, t, PSD = signal.spectrogram(resid_T, fs, window=('hann'), nperseg=None, mode='psd')
# import matplotlib.pyplot as plt
# plt.pcolormesh(t, f, PSD, shading='gouraud')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

## Periodogram

In [ ]:
# f, Pxx_den = signal.periodogram(resid_T,fs)
# plt.semilogy(f, Pxx_den)
# plt.ylim([1e-7, 1e2])
# plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
# plt.show()

## Envelope Function:


In [ ]:
# from numpy import array, sign, zeros
# from scipy.interpolate import interp1d

# s = resid_T

# q_u = zeros(s.shape)
# q_l = zeros(s.shape)

# #Prepend the first value of (s) to the interpolating values. 
# #This forces the model to use the same starting point for both the upper and lower envelope models.
# u_x = [0,]
# u_y = [s[0],]

# l_x = [0,]
# l_y = [s[0],]

# #Detect peaks and troughs and mark their location in u_x,u_y,l_x,l_y respectively.

# for k in range(1,len(s)-1):
#     if (sign(s[k]-s[k-1])==1) and (sign(s[k]-s[k+1])==1):
#         u_x.append(k)
#         u_y.append(s[k])

#     if (sign(s[k]-s[k-1])==-1) and ((sign(s[k]-s[k+1]))==-1):
#         l_x.append(k)
#         l_y.append(s[k])

# #Append the last value of (s) to the interpolating values. This forces the model to use the same ending point for both the upper and lower envelope models.

# u_x.append(len(s)-1)
# u_y.append(s[-1])

# l_x.append(len(s)-1)
# l_y.append(s[-1])

# #Fit suitable models to the data. Here I am using cubic splines, similarly to the MATLAB example given in the question.

# u_p = interp1d(u_x,u_y, kind = 'cubic',bounds_error = False, fill_value=0.0)
# l_p = interp1d(l_x,l_y,kind = 'cubic',bounds_error = False, fill_value=0.0)

# #Evaluate each model over the domain of (s)
# for k in range(0,len(s)):
#     q_u[k] = u_p(k)
#     q_l[k] = l_p(k)

# #Plot everything
# plt.plot(s)
# plt.plot(q_u,'r')
# plt.plot(q_l,'g')
# plt.grid(True)
# plt.show()


import numpy as np
from matplotlib import pyplot as plt

def hl_envelopes_idx(s, dmin=1, dmax=1, split=False):
    """
    Input :
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases
    Output :
    lmin,lmax : high/low envelope idx of input signal s
    """

    # locals min      
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    # locals max
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
    

    if split:
        # s_mid is zero if s centered around x-axis or more generally mean of signal
        s_mid = np.mean(s) 
        # pre-sorting of locals min based on relative position with respect to s_mid 
        lmin = lmin[s[lmin]<s_mid]
        # pre-sorting of local max based on relative position with respect to s_mid 
        lmax = lmax[s[lmax]>s_mid]


    # global max of dmax-chunks of locals max 
    lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
    # global min of dmin-chunks of locals min 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
    
    return lmin,lmax


t = C_1['Date_pd']
s = resids['T']
high_idx, low_idx = hl_envelopes_idx(s)

# plot
plt.plot(t,s,label='signal')
plt.plot(t[high_idx], s[high_idx], 'r', label='low')
plt.plot(t[low_idx], s[low_idx], 'g', label='high')
plt.xlabel('Date')
plt.ylabel('Residual (m)')
plt.xticks(rotation = 45) 
plt.show()

## Residual Analysis


- what are these residuals?
    - the orbit is a modeled fit to the data
- detrend if necessary
    - do an FFT
    - plot periodogram
- find most dominant frequencies
- we COULD make a least squares fit to the data   (what is advantage of this)
- look at histogram of the residuals
    - is it normal?




### Detrend if necessary


In [ ]:
# #### detrend if necessary
# plt.title('Not necessary to detrend the In Trackdata')
# plt.plot(C_1['Date_pd'], resid_T)
# plt.xlabel('Date')
# plt.ylabel('Resdual (m)')
# plt.xticks(rotation = 45) 
# plt.show()



### Do an FFT and Periodogram


In [ ]:

# #### do an FFT
# # %do fft
# # A_fft_det = fft(A_det);
# # %plot periodogram
# # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # Fs = 1
# #  % this may need to change
# # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # N_A_det = length(A_det);
# # A_fft_det = A_fft_det(1:N_A_det/2+1);
# # psd_A_det = (1/(Fs*N_A_det)) * abs(A_fft_det).^2;
# # psd_A_det(2:end-1) = 2*psd_A_det(2:end-1);
# # freq_A_det = 0:Fs/length(A_det):Fs/2;
# # figure(2)
# # plot(freq_A_det,(psd_A_det))
# # grid on
# # title('Periodogram Using FFT for A after Detrending')
# # xlabel('Frequency (Hz)')
# # ylabel('Power/Frequency (dB/Hz)')

# # C_1['Date_pd']
# # resid_T


# from scipy import fft
# resids_fft = fft.fft(resid_T)

# Fs = 1/16
# N = len(resid_T)
# slice_half = int(N/2+1)
# resids_fft = resids_fft[0:slice_half] 

# psd = (1/(Fs*N)) * np.square(np.abs(resids_fft))
#                 # psd = 2*psd[1:-1]
# psd = psd[:-1]
# freq_resids = np.arange(0,Fs/2,Fs/len(resid_T) )
# # freq_A_det = 0:Fs/length(A_det):Fs/2;

# plt.title('Periodogram of the Residuals')
# plt.semilogx(freq_resids, psd, '.-')
# plt.xlabel('Frequency')
# plt.ylabel('power')
# plt.xticks(rotation = 45) 
# plt.show()

####  NTW

In [ ]:
# coords = ['N','T','W']
# ms_size = 5

# fig = make_subplots(
#     rows=3, cols=1,
#     subplot_titles=(coords),
#     vertical_spacing = 0.2)
# fig.update_layout(title=  'Residuals '+  ''.join(coords)+' System',
#                 autosize=True,
#                 font=dict(size=14),
#                 legend= {'itemsizing': 'constant'})

# for i,val in enumerate(coords):
#     fig.add_trace(go.Scattergl(
#                              x=resids['Date'],
#                              y=resids[val],
#                          name=val+' resids',
#                          mode='markers',
#                          marker=dict(
#                              color=col1,
#                              size=ms_size,
#                                     ),
#                          showlegend=True),
#                          secondary_y=False,
#                          row=i+1, col=1)

# fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=1, col=1)
# fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=2, col=1)
# fig.update_yaxes( title="meters" ,type="linear" , exponentformat= 'power',row=3, col=1)
# fig.update_xaxes( title="Date"    ,row=3, col=1)

# fig.show(config=config)


In [ ]:
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['FFT of NTW Residuals']),
    vertical_spacing = 0.1)

for comp in ['N', 'T', 'W']:

    t = []
    for i,val in enumerate(resids['Date']):
        t.append(resids['Date'][i].timestamp())

    from scipy.fft import fft, fftfreq
    from scipy.signal import detrend
    
    # Number of sample points
    N = int(np.size(resids[comp]))                       #600

    # sample spacing
    T  = 1/16                                               #1.0 / 800.0
    x  = t                                                  #np.linspace(0.0, N*T, N, endpoint=False)
    y  = detrend(resids[comp])                                   #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]


    fig.add_trace(go.Scattergl(
                             x=xf,
                             y=2.0/N * np.abs(yf[0:N//2]),
                         name=comp+' fft',
                         mode='markers+lines',
                         marker=dict(
#                              color=col1,
                                 size=6,
                                    ),
                         showlegend=True,
                         ),
                         secondary_y=False,
                         row=1, col=1,
                         )

fig.update_yaxes( title="Power"    ,type="linear",exponentformat= 'power',row=1, col=1)
fig.update_xaxes( title="Frequency", type="log",row=1, col=1)

fig.show(config=config)


####  XYZ 

In [ ]:
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['FFT of XYZ Residuals']),
    vertical_spacing = 0.1)

for comp in ['X', 'Y', 'Z']:

    t = []
    for i,val in enumerate(resids['Date']):
        t.append(resids['Date'][i].timestamp())

    from scipy.fft import fft, fftfreq
    from scipy.signal import detrend
    
    # Number of sample points
    N = int(np.size(resids[comp]))                       #600

    # sample spacing
    T  = 1/16                                               #1.0 / 800.0
    x  = t                                                  #np.linspace(0.0, N*T, N, endpoint=False)
    y  = detrend(resids[comp])                                   #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]


    fig.add_trace(go.Scattergl(
                             x=xf,
                             y=2.0/N * np.abs(yf[0:N//2]),
                         name=comp+' fft',
                         mode='markers+lines',
                         marker=dict(
#                              color=col1,
                                 size=6,
                                    ),
                         showlegend=True,
                         ),
                         secondary_y=False,
                         row=1, col=1,
                         )

fig.update_yaxes( title="Power"    ,type="linear",exponentformat= 'power',row=1, col=1)
fig.update_xaxes( title="Frequency", type="log",row=1, col=1)

fig.show(config=config)


In [ ]:
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['FFT of XYZ Component Residuals']),
    vertical_spacing = 0.1)


index_pce_x = obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE X    '
index_pce_y = obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE Y    '
index_pce_z = obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE Z    '
# date_x    = obj_m1.Residuals_obs[arc]['Date'][index_pce_x]
resids_x  = obj_m1.Residuals_obs[arc]['Residual'][index_pce_x].values
# date_y    = obj_m1.Residuals_obs[arc]['Date'][index_pce_y]
resids_y  = obj_m1.Residuals_obs[arc]['Residual'][index_pce_y].values
# date_z    = obj_m1.Residuals_obs[arc]['Date'][index_pce_z]
resids_z  = obj_m1.Residuals_obs[arc]['Residual'][index_pce_z].values


for comp in ['X', 'Y', 'Z']:
    
    
    date    = pd.to_datetime(obj_m1.Residuals_obs[arc]['Date'][obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE '+comp+'    ']).reset_index()
    resids_pce  = obj_m1.Residuals_obs[arc]['Residual'][obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE '+comp+'    '].values

    
#     t = []
#     for i,val in enumerate(date['Date']):
#         t.append(date['Date'][i].timestamp())
    
    from scipy.fft import fft, fftfreq


    # Number of sample points
    N = int(np.size(resids_pce))                              #600

    # sample spacing
    T  = 1                                               #1.0 / 800.0
#     x  = t                                                  #np.linspace(0.0, N*T, N, endpoint=False)
    y  = resids_pce                                            #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]



    fig.add_trace(go.Scattergl(
                             x=xf,
                             y=2.0/N * np.abs(yf[0:N//2]),
                         name='PCE'+comp,
                         mode='markers+lines',
                         marker=dict(  #color=col1,
                                 size=6,
                                    ),
                         showlegend=True,
                         ),
                         secondary_y=False,
                         row=1, col=1,
                         )


    
date    = pd.to_datetime(obj_m1.Residuals_obs[arc]['Date'][obj_m1.Residuals_obs[arc]['StatSatConfig'] == 'PCE '+'X'+'    ']).reset_index()
resids_pce  = np.sqrt( np.square(resids_x) + np.square(resids_y) + np.square(resids_z)  )
# t = []
# for i,val in enumerate(date['Date']):
#     t.append(date['Date'][i].timestamp())
# Number of sample points
N = int(np.size(resids_pce))                              #600

# sample spacing
T  = 1                                               #1.0 / 800.0
# x  = t                                                  #np.linspace(0.0, N*T, N, endpoint=False)
y  = resids_pce                                            #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
yf = fft(y)
xf = fftfreq(N, T)[:N//2]

fig.add_trace(go.Scattergl(
                         x=xf,
                         y=2.0/N * np.abs(yf[0:N//2]),
                     name='Total Vector',
                     mode='markers+lines',
                     marker=dict(color='black',
                             size=6,
                                ),
                     showlegend=True,
                     ),
                     secondary_y=False,
                     row=1, col=1,
                     )

    

    
fig.update_yaxes( title="Power"    , type="linear",exponentformat= 'power',row=1, col=1)
fig.update_xaxes( title="Frequency", type="log",   exponentformat= 'power',row=1, col=1)

fig.show(config=config)


#### R theta phi

In [ ]:
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['FFT of R theta phi Residuals']),
    vertical_spacing = 0.1)

for comp in ['R', 'theta', 'phi']:

    t = []
    for i,val in enumerate(resids['Date']):
        t.append(resids['Date'][i].timestamp())

    from scipy.fft import fft, fftfreq
    from scipy.signal import detrend
    
    # Number of sample points
    N = int(np.size(resids[comp]))                       #600

    # sample spacing
    T  = 1/16                                               #1.0 / 800.0
    x  = t                                                  #np.linspace(0.0, N*T, N, endpoint=False)
    y  = detrend(resids[comp])                                   #np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]


    fig.add_trace(go.Scattergl(
                             x=xf,
                             y=2.0/N * np.abs(yf[0:N//2]),
                         name=comp+' fft',
                         mode='markers+lines',
                         marker=dict(
#                              color=col1,
                                 size=6,
                                    ),
                         showlegend=True,
                         ),
                         secondary_y=False,
                         row=1, col=1,
                         )

fig.update_yaxes( title="Power"    , type="linear",exponentformat= 'power',row=1, col=1)
fig.update_xaxes( title="Frequency", type="log",row=1, col=1)

fig.show(config=config)


### find most dominant frequencies

In [ ]:
#### find most dominant frequencies
# freq_resids == np.max(freq_resids)


# print(domfreq_1)
# filtered_1 = psd_A_det;
# for i = 1:length(filtered_1)
# if filtered_1(i) == max(psd_A_det)
# filtered_1(i) = 0;
# end
# end
# domfreq_A2 = freq_A_det(find(filtered_1 == max(filtered_1)))


### look at histogram of the residual.
is it normal?  What is the distrubution?

In [ ]:
range(-3, 3, 10)

In [ ]:
#### look at histogram of the residual. is it normal?  What is the distrubution?

import scipy
import numpy as np 
import matplotlib.pyplot as plt
  
hist, bin_edges = np.histogram([resid_T],
                                       bins = np.linspace(-3,3, 20))
  
# Checking the results
print ("No. of points in each bin : ", hist)
print ("Size of the bins          : ", bin_edges)
  
# plotting the histogram
plt.bar(bin_edges[:-1], hist, width = 1)
plt.xlim(min(bin_edges), max(bin_edges))
plt.show()

### we COULD make a least squares fit to the data  (what is advantage of this)

In [ ]:
#### we COULD make a least squares fit to the data  (what is advantage of this)
